In [4]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 23.1 MB/s eta 0:00:00


In [10]:
!wget https://github.com/Educat8n/images/raw/refs/heads/main/dogs.mp4

--2025-04-08 10:31:10--  https://github.com/Educat8n/images/raw/refs/heads/main/dogs.mp4
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Educat8n/images/refs/heads/main/dogs.mp4 [following]
--2025-04-08 10:31:10--  https://raw.githubusercontent.com/Educat8n/images/refs/heads/main/dogs.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4913572 (4.7M) [application/octet-stream]
Saving to: ‘dogs.mp4’

dogs.mp4            100%[===================>]   4.69M  --.-KB/s    in 0.09s   

2025-04-08 10:31:11 (54.1 MB/s) - ‘dogs.mp4’ saved [4913572/4913572]



In [11]:
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from torchvision.io import read_video
from torchvision.transforms import Compose, Resize, CenterCrop, ToPILImage
import torch

# Load model and processor
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")
processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")

# Load video (fix timestamp warning)
video_path = "dogs.mp4"
video, _, _ = read_video(video_path, pts_unit='sec')

# Use first 8 frames
frames = video[:8]  # shape: [T, H, W, C]

# Define transform: convert to PIL (let processor handle scaling)
transform = Compose([
    Resize(256),
    CenterCrop(224),
    ToPILImage()
])

# Transform each frame to PIL
processed_frames = [transform(frame.permute(2, 0, 1)) for frame in frames]

# Pass through processor
inputs = processor(images=processed_frames, return_tensors="pt", do_rescale=False)

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Print top prediction
pred_class = outputs.logits.argmax(-1).item()
print("Predicted action:", model.config.id2label[pred_class])


Predicted action: carrying baby
